# Project 2: Supervised Learning
### Building a Student Intervention System

## 1. Classification vs Regression

Your goal is to identify students who might need early intervention - which type of supervised machine learning problem is this, classification or regression? Why?



## 2. Exploring the Data

Let's go ahead and read in the student dataset first.

_To execute a code cell, click inside it and press **Shift+Enter**._

In [13]:
# Import libraries
import numpy as np
import pandas as pd

In [14]:
# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"
# Note: The last column 'passed' is the target/label, all other are feature columns

Student data read successfully!


Now, can you find out the following facts about the dataset?
- Total number of students
- Number of students who passed
- Number of students who failed
- Graduation rate of the class (%)
- Number of features

_Use the code block below to compute these values. Instructions/steps are marked using **TODO**s._

In [15]:
# TODO: Compute desired values - replace each '?' with an appropriate expression/function call
n_students = len(student_data)
n_features = student_data.shape[1]-1 #Subtract  the target label column
n_passed = len(student_data.query("passed == 'yes'"))
n_failed = len(student_data.query("passed == 'no'"))
grad_rate = float(n_passed)/n_students*100
print "Total number of students: {}".format(n_students)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Number of features: {}".format(n_features)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of students who passed: 265
Number of students who failed: 130
Number of features: 30
Graduation rate of the class: 67.09%


## 3. Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Let's first separate our data into feature and target columns, and see if any features are non-numeric.<br/>
**Note**: For this dataset, the last column (`'passed'`) is the target or label we are trying to predict.

In [16]:
# Extract feature (X) and target (y) columns
feature_cols = list(student_data.columns[:-1])  # all columns but last are features
target_col = student_data.columns[-1]  # last column is the target/label
print "Feature column(s):-\n{}".format(feature_cols)
print "Target column: {}".format(target_col)

X_all = student_data[feature_cols]  # feature values for all students
y_all = student_data[target_col]  # corresponding targets/labels
print "\nFeature values:-"
print X_all.head()  # print the first 5 rows

Feature column(s):-
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']
Target column: passed

Feature values:-
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...    

### Preprocess feature columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation.

In [17]:
# Preprocess feature columns
def preprocess_features(X):
    outX = pd.DataFrame(index=X.index)  # output dataframe, initially empty

    # Check each column
    for col, col_data in X.iteritems():
        # If data type is non-numeric, try to replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])
        # Note: This should change the data type for yes/no columns to int

        # If still non-numeric, convert to one or more dummy variables
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix=col)  # e.g. 'school' => 'school_GP', 'school_MS'

        outX = outX.join(col_data)  # collect column(s) in output dataframe

    return outX

X_all = preprocess_features(X_all)
print "Processed feature columns ({}):-\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48):-
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Split data into training and test sets

So far, we have converted all _categorical_ features into numeric values. In this next step, we split the data (both features and corresponding labels) into training and test sets.

In [18]:
# First, decide how many training vs test samples you want
num_all = student_data.shape[0]  # same as len(student_data)
num_train = 300  # about 75% of the data
num_test = num_all - num_train

# TODO: Then, select features (X) and corresponding labels (y) for the training and test sets
# Note: Shuffle the data or randomly select samples to avoid any bias due to ordering in the dataset

from sklearn.cross_validation import train_test_split
split_data = train_test_split( X_all, y_all, test_size=num_test)
X_train = split_data[0]
y_train = split_data[2]
X_test = split_data[1]
y_test = split_data[3]
print "Training set: {} samples".format(X_train.shape[0])
print "Test set: {} samples".format(X_test.shape[0])
# Note: If you need a validation set, extract it from within training data

Training set: 300 samples
Test set: 95 samples


## 4. Training and Evaluating Models
Choose 3 supervised learning models that are available in scikit-learn, and appropriate for this problem. For each model:

- What are the general applications of this model? What are its strengths and weaknesses?
- Given what you know about the data so far, why did you choose this model to apply?
- Fit this model to the training data, try to predict labels (for both training and test sets), and measure the F<sub>1</sub> score. Repeat this process with different training set sizes (100, 200, 300), keeping test set constant.

Produce a table showing training time, prediction time, F<sub>1</sub> score on training set and F<sub>1</sub> score on test set, for each training set size.

Note: You need to produce 3 such tables - one for each model.

In [19]:
# Train a model
import time

def train_classifier(clf, X_train, y_train):
    print "Training {}...".format(clf.__class__.__name__)
    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
    print "Training time (secs): {:.3f}".format(end - start)

# TODO: Choose a model, import it and instantiate an object
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=0)

# from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(C=1.0, penalty='l2' )
# clf = LogisticRegressionCV(penalty='l2', solver='liblinear', Cs=[0.01, 0.001], refit=True)

# Fit model to training data
train_classifier(clf, X_train, y_train)  # note: using entire training set here
#print clf  # you can inspect the learned model by printing it

print clf.coef_
print np.argmax(clf.coef_)
print "Feature column(s):-\n{}".format(X_train.columns)
print "Feature column(s):-\n{}".format(X_train.ix(0,12))

Training LogisticRegression...
Training time (secs): 0.004
[[ 0.35898725  0.05406443  0.0400709   0.37298078 -0.07338158  0.2101692
   0.20288248  0.08973562  0.32331606  0.53906767 -0.126016    0.13530833
   0.06948729  0.06136398  0.53617379 -0.07803703  0.49206364 -0.59851271
   0.05765412 -0.31631855  0.12994739 -0.10635261  0.64812132 -0.21116802
   0.18186482  0.01662963  0.42572524  0.21704538  0.02654527  0.16946102
   0.09083398  0.20666743 -1.04191421 -0.93756948 -0.28336009  0.177322
  -0.23800564 -0.5188507   0.46326848  0.25214068 -0.16573268  0.21856531
   0.10488955 -0.36929252 -0.062811    0.19996456 -0.12519123 -0.01934296]]
22
Feature column(s):-
Index([u'school_GP', u'school_MS', u'sex_F', u'sex_M', u'age', u'address_R',
       u'address_U', u'famsize_GT3', u'famsize_LE3', u'Pstatus_A',
       u'Pstatus_T', u'Medu', u'Fedu', u'Mjob_at_home', u'Mjob_health',
       u'Mjob_other', u'Mjob_services', u'Mjob_teacher', u'Fjob_at_home',
       u'Fjob_health', u'Fjob_other',

In [20]:
# Predict on training set and compute F1 score
from sklearn.metrics import f1_score

def predict_labels(clf, features, target):
    print "Predicting labels using {}...".format(clf.__class__.__name__)
    start = time.time()
    y_pred = clf.predict(features)
    end = time.time()
    print "Prediction time (secs): {:.3f}".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')

train_f1_score = predict_labels(clf, X_train, y_train)
print "F1 score for training set: {}".format(train_f1_score)

Predicting labels using LogisticRegression...
Prediction time (secs): 0.000
F1 score for training set: 0.835266821346


In [21]:
# Predict on test data
print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

Predicting labels using LogisticRegression...
Prediction time (secs): 0.000
F1 score for test set: 0.788732394366


In [22]:
# Train and predict using different training set sizes
def train_predict(clf, X_train, y_train, X_test, y_test):
    print "------------------------------------------"
    print "Training set size: {}".format(len(X_train))
    train_classifier(clf, X_train, y_train)
    print "F1 score for training set: {}".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

# TODO: Run the helper function above for desired subsets of training data
# Note: Keep the test set constant


# Train and predict with training set sizes 100, 200, 300. test size is 95.
def train_predict_on_three_training_sets(clf, X_all, y_all):
    for i in [100,200,300]:
        train_predict(clf, X_train[:i], y_train[:i], X_test, y_test)

train_predict_on_three_training_sets(clf, X_all, y_all)


------------------------------------------
Training set size: 100
Training LogisticRegression...
Training time (secs): 0.001
Predicting labels using LogisticRegression...
Prediction time (secs): 0.000
F1 score for training set: 0.90780141844
Predicting labels using LogisticRegression...
Prediction time (secs): 0.000
F1 score for test set: 0.811594202899
------------------------------------------
Training set size: 200
Training LogisticRegression...
Training time (secs): 0.002
Predicting labels using LogisticRegression...
Prediction time (secs): 0.000
F1 score for training set: 0.861111111111
Predicting labels using LogisticRegression...
Prediction time (secs): 0.000
F1 score for test set: 0.819444444444
------------------------------------------
Training set size: 300
Training LogisticRegression...
Training time (secs): 0.003
Predicting labels using LogisticRegression...
Prediction time (secs): 0.000
F1 score for training set: 0.835266821346
Predicting labels using LogisticRegression..

In [23]:
# TODO: Train and predict using two other models
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
clfs = [
    LogisticRegression( ),
    SVC(),
    RandomForestClassifier()
    ]

for clf in clfs:
    train_predict_on_three_training_sets(clf, X_all, y_all)

------------------------------------------
Training set size: 100
Training LogisticRegression...
Training time (secs): 0.002
Predicting labels using LogisticRegression...
Prediction time (secs): 0.000
F1 score for training set: 0.90780141844
Predicting labels using LogisticRegression...
Prediction time (secs): 0.000
F1 score for test set: 0.811594202899
------------------------------------------
Training set size: 200
Training LogisticRegression...
Training time (secs): 0.002
Predicting labels using LogisticRegression...
Prediction time (secs): 0.000
F1 score for training set: 0.861111111111
Predicting labels using LogisticRegression...
Prediction time (secs): 0.000
F1 score for test set: 0.819444444444
------------------------------------------
Training set size: 300
Training LogisticRegression...
Training time (secs): 0.002
Predicting labels using LogisticRegression...
Prediction time (secs): 0.000
F1 score for training set: 0.835266821346
Predicting labels using LogisticRegression..

## 5. Choosing the Best Model

- Based on the experiments you performed earlier, in 1-2 paragraphs explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?
- In 1-2 paragraphs explain to the board of supervisors in layman's terms how the final model chosen is supposed to work (for example if you chose a Decision Tree or Support Vector Machine, how does it make a prediction).
- Fine-tune the model. Use Gridsearch with at least one important parameter tuned and with at least 3 settings. Use the entire training set for this.
- What is the model's final F<sub>1</sub> score?

In [24]:
# TODO: Fine-tune your model and report the best F1 score

from sklearn.metrics import make_scorer
def my_performance_metric(label, prediction):
    return f1_score(label, prediction, pos_label='yes')

my_scorer_func = make_scorer(my_performance_metric, greater_is_better=True)


from sklearn.grid_search import GridSearchCV
from sklearn import cross_validation
clf = LogisticRegression(solver='liblinear', dual=True, penalty='l2', n_jobs=-1)


C_range = np.logspace(-2, 2, 100)
param_grid = dict(C=C_range)

cv = cross_validation.StratifiedShuffleSplit(y_train, n_iter=5, test_size=0.8, random_state=None)
grid = GridSearchCV(clf, param_grid=param_grid, cv=cv, scoring=my_scorer_func)
grid.fit(X_train, y_train)

print("The best parameters are %s with a score of %0.2f"
      % (grid.best_params_, grid.best_score_))

print "F1 score for test set: {}".format(predict_labels(grid, X_test, y_test))

print "{}".format(grid.__class__.__name__)
scores= cross_validation.cross_val_score(grid, X_all, y_all, cv=5, scoring=my_scorer_func, n_jobs=-1)
print "F1 score: {}".format(np.average(scores))



The best parameters are {'C': 0.014508287784959394} with a score of 0.80
Predicting labels using GridSearchCV...
Prediction time (secs): 0.000
F1 score for test set: 0.822784810127
GridSearchCV
F1 score: 0.813893966379
